#### Environment Setup

In [3]:
TChem_install_directory ='where/TChem/is/installed'

In [4]:
import sys
sys.path.append(TChem_install_directory+'/lib')
import pytchem
import numpy as np
import matplotlib.pyplot as plt
import time

In [5]:
# Pressure, Temperature
Pressure = 101325 # Pa 
Temperature = 1000 # K 

Variables = ['T','P','CH4','O2','N2']
Nsamples = 100
Nvar = len(Variables)
sample = np.zeros([Nsamples,Nvar])
sample[:,0] =  Temperature
sample[:,1] =  Pressure
sample[:,2] =  0.1 ## CH4
sample[:,3] =  0.2 ## O2
sample[:,4] =  0.7 ## N2

## TChem Simulation

### Initialize TChem Driver Object

* Initialization of Kokkos.
* Create a TChem driver object. 

In [6]:
pytchem.initialize()

In [7]:
tchem = pytchem.TChemDriver()

* Get help from TChem driver object.

In [8]:
help(tchem)

Help on TChemDriver in module pytchem object:

class TChemDriver(pybind11_builtins.pybind11_object)
 |  A class to manage data movement between numpy to kokkos views in TChem::Driver object
 |  
 |  Method resolution order:
 |      TChemDriver
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(self: pytchem.TChemDriver) -> None
 |  
 |  cloneGasKineticModel(...)
 |      cloneGasKineticModel(self: pytchem.TChemDriver) -> None
 |      
 |      Internally create clones of the kinetic model
 |  
 |  computeGasEnthapyMass(...)
 |      computeGasEnthapyMass(self: pytchem.TChemDriver) -> None
 |      
 |      Compute enthalpy mass and mixture enthalpy
 |  
 |  computeGasNetProductionRatePerMass(...)
 |      computeGasNetProductionRatePerMass(self: pytchem.TChemDriver) -> None
 |      
 |      Compute net production rate
 |  
 |  computeGasReactionRateConstants(...)
 |      computeGasReactionRateConstants(sel

### Create Kinetic Model 

* Inputs are the reactions mechanism files; in this case, we use the GRI3.0 gas reaction mechanism

In [9]:
inputs_directory = TChem_install_directory + '/example/data/ignition-zero-d/gri3.0/'
tchem.createGasKineticModel(inputs_directory+'chem.inp',inputs_directory+'therm.dat')

* Set number of samples
* Internally construct const object of the kinetic model and load them to device

In [10]:
tchem.setNumberOfSamples(Nsamples)
tchem.createGasKineticModelConstData()

### Set State Vector

* Get index for variables. 
* Pass a numpy array to the TChem object to set the state vector. 

In [11]:
indx=[]
for var in Variables:
    indx += [tchem.getStateVariableIndex(var)]

state = np.zeros([Nsamples, tchem.getLengthOfStateVector()])
for sp in range(Nsamples):
    state[sp,indx] = sample[sp,:]

In [12]:
tchem.setNumberOfSamples(Nsamples)
tchem.createStateVector()
tchem.setStateVector(state)

### Compute net production rate in kg/s
$$
\dot{\omega}_k=W_k\sum_{i=1}^{N_{reac}}\nu_{ki}q_i,\,\,\, \nu_{ki}=\nu''_{ki}-\nu'_{ki},
$$

In [13]:
tchem.computeGasNetProductionRatePerMass()

In [14]:
net_production_rate = tchem.getGasNetProductionRatePerMass()

In [15]:
print('number of samples and number of species', np.shape(net_production_rate))
print(net_production_rate)

number of samples and number of species (100, 53)
[[0.00000000e+00 7.81459512e-11 1.15813498e-16 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.81459512e-11 1.15813498e-16 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.81459512e-11 1.15813498e-16 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 7.81459512e-11 1.15813498e-16 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.81459512e-11 1.15813498e-16 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.81459512e-11 1.15813498e-16 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


Finalize Kokkos. This deletes the TChem object and the data stored as Kokkos views

In [16]:
del(tchem)

In [17]:
pytchem.finalize()